In [2]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize 

import pickle
import collections 

In [191]:
def sanitize(text):
    text = text.lower()
    return ' '.join(word_tokenize(text)[:-1])

In [6]:
data_path = '/home/vk352/paraphraseDomainShift/data/'
spm_path = '/home/vk352/paraphraseDomainShift/'

In [192]:
# quora data
data_quora = pd.read_csv(data_path+'quora_duplicate_questions.tsv', sep='\t')
data_quora = data_quora[['question1', 'question2', 'is_duplicate']]
data_quora.rename(columns={'is_duplicate': 'paraphrase'}, inplace=True)
data_quora.dropna(inplace=True)
data_quora['question1'] = data_quora['question1'].apply(lambda text: sanitize(text))
data_quora['question2'] = data_quora['question2'].apply(lambda text: sanitize(text))
# data_quora.to_pickle('data_quora')

In [193]:
data_quora.to_pickle(data_path+'data_quora')

In [256]:
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/train/a.toks', train['question1'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/train/a.toks', train['question2'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/train/a.toks', train['paraphrase'].values, fmt='%s')

np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/test/a.toks', test['question1'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/test/b.toks', test['question2'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/test/sim.txt', test['paraphrase'].values, fmt='%s')

np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/dev/a.toks', dev['question1'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/dev/b.toks', dev['question2'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/dev/sim.txt', dev['paraphrase'].values, fmt='%s')

In [244]:
data_quora = pd.read_pickle(data_path+'data_quora')
test = data_quora[-10000:]
dev = data_quora[:-10000][-10000:]
train = data_quora[:-20000]

In [281]:
# msr_data 
with open(data_path+'msr-paraphrase-corpus/msr_paraphrase_train.txt', 'r') as file:
    lines = file.readlines() 
data_msr = pd.DataFrame(data={'utt1': [line.split('\t')[3] for line in lines[1:]],
                             'utt2': [line.split('\t')[4] for line in lines[1:]],
                             'paraphrase': [line.split('\t')[0] for line in lines[1:]]})
with open(data_path+'msr-paraphrase-corpus/msr_paraphrase_test.txt', 'r') as file:
    lines = file.readlines() 
data_msr_test = pd.DataFrame(data={'utt1': [line.split('\t')[3] for line in lines[1:]],
                             'utt2': [line.split('\t')[4] for line in lines[1:]],
                             'paraphrase': [line.split('\t')[0] for line in lines[1:]]})

In [285]:
data_msr['utt1'] = data_msr['utt1'].apply(lambda text: sanitize(text))
data_msr['utt2'] = data_msr['utt2'].apply(lambda text: sanitize(text))
data_msr_test['utt1'] = data_msr_test['utt1'].apply(lambda text: sanitize(text))
data_msr_test['utt2'] = data_msr_test['utt2'].apply(lambda text: sanitize(text))
split = int(0.8*len(data_msr))
data_msr_train = data_msr[:split]
data_msr_val = data_msr[split:]

np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/train/a.toks', data_msr_train['utt1'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/train/b.toks', data_msr_train['utt2'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/train/sim.txt', data_msr_train['paraphrase'].values, fmt='%s')

np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/test/a.toks', data_msr_test['utt1'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/test/b.toks', data_msr_test['utt2'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/test/sim.txt', data_msr_test['paraphrase'].values, fmt='%s')

np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/dev/a.toks', data_msr_val['utt1'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/dev/b.toks', data_msr_val['utt2'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/dev/sim.txt', data_msr_val['paraphrase'].values, fmt='%s')

In [349]:
li = []
from operator import itemgetter
for sent in list(data_msr['utt1'])+list(data_msr['utt2']):
    sent = sent.lower()
    li.extend(word_tokenize(sent)[:-1])
li = set(li)
d = {}
for i, elem in enumerate(li):
    d[elem] = i
# di = collections.OrderedDict(sorted(d.items(), key=itemgetter(1)))
# pickle.dump( di, open( "msr_vocab_cased.pkl", "wb" ) )

In [350]:
# d = {'banana': 3, 'apple':4, 'pear': 1, 'orange': 2}
sorted_dict = collections.OrderedDict(sorted(d.items(), key=lambda t: t[1]))

In [353]:
len(sorted_dict), len(li)

(13008, 13008)

In [287]:
#twitter data
data_twitter = pd.read_csv(data_path+'twitter/Twitter_URL_Corpus_train.txt', sep='\t', header=None)
data_twitter.columns = ['utt1', 'utt2', 'paraphrase', 'link']
data_twitter = data_twitter[['utt1', 'utt2', 'paraphrase']]
data_twitter['paraphrase'] = data_twitter['paraphrase'].apply(lambda x: int(re.search('(.),', x).group(1)))
# original paper removes entries where annotators are split
data_twitter = data_twitter[data_twitter['paraphrase']!=3] 
data_twitter['paraphrase'] = data_twitter['paraphrase'].apply(lambda x: '1' if x>=4 else '0')

data_twitter_test = pd.read_csv(data_path+'twitter/Twitter_URL_Corpus_test.txt', sep='\t', header=None)
data_twitter_test.columns = ['utt1', 'utt2', 'paraphrase', 'link']
data_twitter_test = data_twitter_test[['utt1', 'utt2', 'paraphrase']]
data_twitter_test['paraphrase'] = data_twitter_test['paraphrase'].apply(lambda x: int(re.search('(.),', x).group(1)))
# original paper removes entries where annotators are split
data_twitter_test = data_twitter_test[data_twitter_test['paraphrase']!=3] 
data_twitter_test['paraphrase'] = data_twitter_test['paraphrase'].apply(lambda x: '1' if x>=4 else '0')

In [288]:
data_twitter['utt1'] = data_twitter['utt1'].apply(lambda text: sanitize(text))
data_twitter['utt2'] = data_twitter['utt2'].apply(lambda text: sanitize(text))
data_twitter_test['utt1'] = data_twitter_test['utt1'].apply(lambda text: sanitize(text))
data_twitter_test['utt2'] = data_twitter_test['utt2'].apply(lambda text: sanitize(text))
split = int(0.8*len(data_twitter))
data_twitter_train = data_twitter[:split]
data_twitter_val = data_twitter[split:]

np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/train/a.toks', data_twitter_train['utt1'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/train/b.toks', data_twitter_train['utt2'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/train/sim.txt', data_twitter_train['paraphrase'].values, fmt='%s')

np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/test/a.toks', data_twitter_test['utt1'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/test/b.toks', data_twitter_test['utt2'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/test/sim.txt', data_twitter_test['paraphrase'].values, fmt='%s')

np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/dev/a.toks', data_twitter_val['utt1'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/dev/b.toks', data_twitter_val['utt2'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/dev/sim.txt', data_twitter_val['paraphrase'].values, fmt='%s')

In [10]:
# Paws qqp
data_paws_qqp = pd.read_csv(data_path+'PAWS/paws_qqp/train.tsv', sep='\t')
data_paws_qqp_dev_test = pd.read_csv(data_path+'PAWS/paws_qqp/dev_and_test.tsv', sep='\t')

In [12]:
# paws
data_paws_train = pd.read_csv(data_path+'PAWS/final/train.tsv', sep='\t')
data_paws_test = pd.read_csv(data_path+'PAWS/final/test.tsv', sep='\t')
data_paws_dev = pd.read_csv(data_path+'PAWS/final/dev.tsv', sep='\t')

In [13]:
len(data_paws_train), len(data_paws_dev), len(data_paws_test)

(49401, 8000, 8000)